# Working with AMBench data in SciServer

## Find, retrieve, and process image data for materials science

#### Version 5

#### This notebook executes a sample query to the AMBench metadata repository via its API, then retrieves and processes the records.  The script then performs some basic image analysis and plots the results.  

#### NOTE: The purpose of the notebook is to show a workflow that uses scripted access (via Python) to obtain and process data; image processing parameters and techniques were selected for this case and are not expected to be generally the best choices for other data or analysis.  Users should perform their own analyses to determine appropriate parameters and techniques for their problems of interest.

The data used in this demonstration comes from the 2018 AM-Bench challenge documented at https://www.nist.gov/ambench/amb2018-02-description.

The script queries the AM-Bench 2018 repository located at https://ambench.nist.gov/, then processes the returned XML-based results, processes the returned images using basic image analysis techniques, and generates plots of melt pool depths.

Additional datasets and challenge problem documentation are available through the NIST Public Data Repository record at https://data.nist.gov/od/id/6D6EC9B3A4147BE2E05324570681EEC91931 with an associated publication (https://link.springer.com/article/10.1007/s40192-020-00169-1) documenting the experiments.

# Setup: prepare the notebook

## Install and import required packages

First, we need to import packages that we will use to do the processing. Each import statement includes a comment describing what that package does.

#### The pycdcs package is used in this notebook to query the AMBench metadata repository that contains information about the AMBench 2018 data.

In [ ]:
# !pip install git+https://github.com/lmhale99/pycdcs
# print('ok')

In [ ]:
# Data processing packages
import pandas
import math
import numpy as np

#image_scale = 0.062   # Image scale in microns per pixel (here, 1 px = 0.062 microns)

# Package to find images in the AMBench repository
## This will allow us to find images to process
## These packages query NIST's Configurable Data Curation System (CDCS):
### https://www.nist.gov/itl/ssd/information-systems-group/configurable-data-curation-system-cdcs
from cdcs import CDCS

# Package to parse metadata from the AMBench repo to find where to download image files
import lxml.etree as et

# Packages to download and save image files
import requests
from urllib import request
import os

# Packages to do image processing once we have the images downloaded
## From the scikit-image package; we only import the parts of the package that we need 
### scikit-image website: https://scikit-image.org/
from skimage import io              # read/write images
from skimage import filters         # apply filters to images
from skimage import segmentation    # divide image into sections
from skimage import morphology    # work with features found in image
from skimage import measure         # measure pixel sizes of features within image

### Packages to plot results
import matplotlib.pyplot as plt   
import matplotlib.image as mpimg  
import matplotlib.patches as patches

pandas.set_option('display.max_colwidth', None)

print('Packages imported!')

## Create functions

For some complex operations that we need to do repeatedly, it's easier to move the operations into a function call. This slows down the code slightly, but makes it much easier to read and to follow the logic of the notebook. 

In [ ]:
# Function to get download links from XML returned by AMBench data repository
# function xml_url_find(xml,searchphrase,mtype):
## Inputs:
### xml: contents of XML file for one of the ambench.nist.gov datasets, in string format
### searchphrase: search phrase contained in the file name (in our case, "BF")
### mtype: filetype (in our case, "image/tiff")

## Returns:
### A four-item list containing:
#### (0) the name of the image file to be downloaded
#### (1) the URL at which to download the image file
#### (2) laser track number for that image
#### (3) case label for that image

def xml_url_find(xml,searchphrase,mtype):
    root=et.fromstring(xml)
    caseid=root.find('.//TraceID')[0].tag[5]
    track=root.find('.//TrackNumber')
    for element in root.iter('downloadURL'):
        u=request.urlopen(element.text)
        if (    # search XML to find searchphrase in filename and mytpe in file type
            (searchphrase in u.info().get_filename()) 
            and (mtype in u.info().get_content_type())
        ):
            name=u.info().get_filename()
            url=element.text
            return [name,url,track.text,caseid]


print('ok')

# 1. Find datasets to work with

The AMBench data repository includes hundreds (thousands?) of datasets associated with challenges. Each dataset includes metadata describing its purpose and various conditions (inputs?) used to generate it. The actual data consists of images (stored as .tiff files) stored on a separate server at NIST. The datasets retreived from the data repository contain pointers to the image locations.

This section of the notebook searches the AMBench data repository for a keyword - in this case, "MP" (for "melt pool"). It then downloads the metadata for all public datasets with your selected keyword. You can see information about each returned dataset by running the code block with the `describe_datasets = True`.

In [ ]:
keyword='MP'                # keyword to search data repository for: MP = melt pool
describe_datasets = True    # show name and basic metadata for each dataset
##show_dataset_xml = False     # parse xml to show full metadata for each dataset

print('querying AMBench data repository for keyword {0:}...'.format(keyword))
curator = CDCS('https://ambench.nist.gov/', username='') # query repo (CDCS) anonymously
datasets_df = curator.query(  # search for results, return as a pandas dataframe:
    keyword=keyword,           ## results should match this keyword
    template='AM-Bench-2018'   ## and be associated with this tempalte
    ) 

### convert all date fields to pandas date format
for thiscol in ['creation_date', 'last_modification_date', 'last_change_date']:
    datasets_df.loc[:, thiscol] = pandas.to_datetime(datasets_df[thiscol], errors='coerce')
    
### use the dataset ID as the index of the dataframe, then sort by that ID    
datasets_df = datasets_df.set_index('id')

print('Found {0:,.0f} datasets matching keyword {1:}!'.format(len(datasets_df), keyword))
print('------------------------------------------------')

if (describe_datasets):
    for ix, thisrow in datasets_df.iterrows():
        print('\n')
        print('Dataset id = {0:}'.format(ix))
        print('\tTitle: {0:}'.format(thisrow['title']))
        print('\tCreated: {0:}'.format(thisrow['creation_date'].strftime('%Y-%m-%d')))
#         if (show_dataset_xml):
#             show_xml(thisrow['xml_content'])
        
print('------------------------------------------------')
print('Done!')

# 2. Find download links for each dataset

Most of the metadata returned by the AMBench data repository is stored in XML format, easy for computers to store and process but hard for humans to read. One important piece of metadata in each dataset is a field called `downloadURL`, which gives the location where image(s) associated with that dataset can be downloaded.

This notebook includes a helper function called `xml_url_find` that searches through the XML to find the download URL for each image to be processed. For this demo notebook, consider that function as a black box; if you want to learn how it works, see the function documentation in the setup step above. The `xml_url_find` function returns track, case, and image URL for each row in the dataset.

Each dataset includes several images (TIFF files), each associated with a specific track and case number. The code cell below loops through each dataset returns these track and case numbers, as well as the URL where the associated image can be downloaded.

In [ ]:
searchphrase='BF'  # used by function xml_url_find, don't worry about what it means now

datasets_df = datasets_df.assign(   # assign blanks to values to be computed later
    filename = np.nan, url = np.nan, track = np.nan, case = np.nan
) 

### Loop through all datasets and get track/case number and URL to download image
print('Finding URLs for images...')
cnt = 0
for ix, thisrow in datasets_df.iterrows():
    if (np.mod(cnt,5) == 0):
        print('\tFinding URL {0:,.0f} of {1:,.0f}...'.format(cnt+1, len(datasets_df)))
    datasets_df.loc[ix, 
                    ['filename', 'url', 'track', 'case']
                   ] = xml_url_find(thisrow['xml_content'],'BF','image/tiff') # giving xml_url_find each xml string and searching for
    cnt += 1


# set track to numeric, then sort by track
datasets_df.loc[:, 'track'] = pandas.to_numeric(
    datasets_df['track'], downcast='integer', errors='coerce'
)
datasets_df = datasets_df.sort_values(by='track') 

# assign blanks for values to compute later
datasets_df = datasets_df.assign(melt_pool_width_pixels = np.nan)
datasets_df = datasets_df.assign(melt_pool_depth_pixels = np.nan)
datasets_df = datasets_df.assign(melt_pool_width_microns = np.nan)
datasets_df = datasets_df.assign(melt_pool_depth_microns = np.nan)


print('\n')
print('Found download links for {0:,.0f} images!'.format(len(datasets_df['url'].dropna())))

datasets_df[['track', 'case', 'url', 'filename', 'melt_pool_width_pixels', 'melt_pool_depth_pixels', 'melt_pool_width_microns', 'melt_pool_depth_microns']]

# 3. Download and process images

This code cell loops through each dataset you found in the previous section and performs the following operations on each (all the image processing is done using the `scikit` image processing package):

1. Downloads the image from the URL you found in the last section
1. Selects a smaller region of the image to work with (to save processing time)
1. Segments that region using the Felzenszwalb algorithm
1. Removes smaller regions (eliminate some background noise)
1. Acquires a table of center coordinates for each remaining region
1. Measures the distances from centers to the center of the entire relevant section image
1. Finds the index number of the closest region
1. Returns information about the closest region
1. Uses those results to measure the width and depth of the melt pool
1. Optionally displays the image with the relevant section marked and measured values displayed

Set `show_images = True` to see each image.

In [ ]:
show_images = True   # set to True to see what the images look like as they are found


print('Getting images...')
cnt = 0
for ix, thisrow in datasets_df.iterrows():
    #if (np.mod(cnt,5) == 0):
    print('\tGetting image {0:,.0f} of {1:,.0f}: track {2:,.0f} case {3:}...'.format(
        cnt+1, len(datasets_df), thisrow['track'], thisrow['case'])
         )
    print(thisrow['url'])
    # read file as an image
    this_image = io.imread(thisrow['url'])
    
    # To save processing time, the segmentation algorithm is applied to only part of the image
    ## The full image is 2832 pixels wide by 4248 pixels high
    ### This code copies and crops the image so that the analysis region is as follows:
    #### rectangle with coordinates 800 <= x <= 3400 , 900 <= y <= 1800
    cropim=this_image[900:1800,800:3400]
    
    # Use Felezenszwalb algorithm to autosegment cropped area
    segments=segmentation.felzenszwalb(cropim,scale=270,sigma=0.8,min_size=1000)

    # Remove small objects, which are likely background noise
    isolateim=morphology.remove_small_objects(segments,100000)

    
    # Find the center point of each segmented area, return as table
    center=measure.regionprops_table(isolateim,properties=['centroid'])
    
    if (show_images):
        center_points = []
        if (len(center['centroid-0']) == len(center['centroid-1'])):
            for i in range(0, len(center['centroid-0'])):
                center_points.append(tuple((center['centroid-1'][i], center['centroid-0'][i])))
    #            print("({0:}, {1:})".format(center['centroid-1'][i], center['centroid-0'][i]))
        else:
            print(len(center['centroid-0']))
            print(len(center['centroid-1']))
        
    # Loop through segments and calculate distance from centers to center of imagebox
    distances=[]
    for n in range(len(center['centroid-0'])):
        distances.append(math.dist([center['centroid-0'][n],center['centroid-1'][n]],[450,1300]))
#    print(distances)
    
        # Choose the region whose center is closest to imagebox center - that's the melt pool
    index=distances.index(min(distances)) #Looking for the region closest to the center of the image - should be the melt pool
    
    object_features=measure.regionprops(isolateim)
    
#    qqdf = pandas.DataFrame(measure.regionprops_table(isolateim))
    
#     for i in range(0, len(object_features)):
#         print(i)
#         print(object_features[i].bbox)
#         print('-----------')
    
    minr,minc,maxr,maxc = object_features[index].bbox
#    print(minr, minc, maxr, maxc)

    

#     print('Melt pool width in pixels: {0:.1f}'.format(maxc-minc))
#     print('\tMelt pool width in microns: {0:.1f}'.format((maxc-minc)*0.062))
    
#     print('Melt pool depth in pixels: {0:.1f}'.format(maxr-minr))
#     print('\tMelt pool depth in microns: {0:.1f}'.format((maxr-minr)*0.062))
    
    datasets_df.loc[ix, 'melt_pool_width_pixels'] = maxc-minc
    datasets_df.loc[ix, 'melt_pool_depth_pixels'] = maxr-minr
    
    datasets_df.loc[ix, 'melt_pool_width_microns'] = (maxc-minc)*0.062
    datasets_df.loc[ix, 'melt_pool_depth_microns'] = (maxr-minr)*0.062
    
    if (show_images):
        print('showing marked image...')
        fig, ax = plt.subplots(1,1,figsize=(16,16/1.5))
        ax.imshow(this_image)
        ax.set_title('Track {0:.0f} case {1:}: full image'.format(thisrow['track'], thisrow['case']))
        ax.set_xlim([0,4248])
        ax.set_xticks(np.arange(0,4300,200))
        ax.set_xlabel('x')
        ax.set_ylim([2832,0])
        ax.set_yticks(np.arange(2800,-200,-200))
        ax.set_ylabel('y')


        rect1 = patches.Rectangle((800,900), 2600, 900, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect1)

#         fig2, ax2 = plt.subplots(1,1,figsize=(16,16/1.5))
#         #ax2.imshow(cropim)
#         ax2.imshow(segmentation.mark_boundaries(cropim, segments))
#         ax2.set_title('Track {0:.0f} case {1:}: cropped segmented image'.format(thisrow['track'], thisrow['case']))
#         ax2.set_xlim([0,2600])
#         ax2.set_xticks(np.arange(0,2700,300))
#         ax2.set_xlabel('x')
#         ax2.set_ylim([900,0])
#         ax2.set_yticks(np.arange(900,-100,-100))
#         ax2.set_ylabel('y')         
        
#        zim = isolateim[17]
        fig3, ax3 = plt.subplots(1,1,figsize=(16,16/1.5))
        #ax2.imshow(cropim)
        ax3.imshow(segmentation.mark_boundaries(cropim, isolateim))
#        ax3.imshow(segmentation.mark_boundaries(cropim, zim, color=(255,0,0)))
        ax3.set_title('Track {0:.0f} case {1:}: cropped segmented isolated image'.format(thisrow['track'], thisrow['case']))
        ax3.set_xlim([0,2600])
        ax3.set_xticks(np.arange(0,2700,300))
        ax3.set_xlabel('x')
        ax3.set_ylim([900,0])
        ax3.set_yticks(np.arange(900,-100,-100))
        ax3.set_ylabel('y')      
        for i in range(0, len(center_points)):
            if (i == index):
                ax3.annotate('{0:.0f}: {1:.1f} px'.format(i, distances[i]), center_points[i], color='red', size=24, va='center', ha='center')
                #ax3.annotate('{0:.0f}: {1:.1f} px'.forma(object_features[i]['label'], distances[i]), center_points[i], color='red', size=24, va='center', ha='center')
            else:
                ax3.annotate('{0:.0f}: {1:.1f} px'.format(i, distances[i]), center_points[i], color='black', size=18, va='center', ha='center')
#                ax3.annotate('{0:.0f}: {1:.1f} px'.format(object_features[i]['label'], distances[i]), center_points[i], color='black', size=18, va='center', ha='center')
        ax3.annotate("X", (1300,450), color='black', size=24, va='center', ha='center')
        plt.show()
#         rect3 = patches.Rectangle((minr,minc), maxr-minr, maxc-minc, linewidth=1, edgecolor='r', facecolor='none')
#         ax3.add_patch(rect3)
    cnt += 1



plt.show()
#qqdf

# Convert track number to integer, add new columns to summary table, sort by track
datasets_df.loc[:, 'track'] = pandas.to_numeric(
    datasets_df['track'], downcast='integer', errors='coerce'
)
datasets_df = datasets_df[
    ['title', 'track', 'case', 'melt_pool_width_pixels', 'melt_pool_depth_pixels', 'melt_pool_width_microns', 'melt_pool_depth_microns', 
     'template', 'workspace',  'creation_date', 'last_modification_date', 
     'last_change_date', 'filename', 'url', 'xml_content', 'template_title', 'user_id']
]
datasets_df = datasets_df.sort_values(by='track')

print('Done!')

# Print melt pool width and depth for each track/case
datasets_df[['track', 'case', 'melt_pool_width_microns', 'melt_pool_depth_microns']]


# for i in range(0, len(isolateim[17])):
#     print(i)
#     print(isolateim[i])
#     print('------')
#print(datasets_df.head(1).T)

## 4. Make plots of melt pool sizes

Now that we have seen

Bar plot of melt pool widths

In [ ]:
datasets_df[['melt_pool_width_microns', 'track']].plot(y='melt_pool_width_microns',x='track',kind='bar')

Bar plot of melt pool depths

In [ ]:
datasets_df[['melt_pool_depth_microns', 'track']].plot(y='melt_pool_depth_microns',x='track',kind='bar')

In [ ]:
datasets_df.plot('melt_pool_depth_microns', 'melt_pool_depth_microns', kind='scatter')

Example of what the image looks like with the bbox marked

In [ ]:
# fig, ax = plt.subplots()
# minr,minc,maxr,maxc=bboxes[4]
# ax.imshow(result[4][900:1800,800:3400])
# bx = (minc, maxc, maxc, minc, minc)
# by = (minr, minr, maxr, maxr, minr)
# ax.plot(bx, by, '-b', linewidth=2.5)
# plt.show()

In [ ]:


# depths=[]
# widths=[]
# bboxes=[]
# for im in dfres['image']:
#     objf=draw_box_opt(im)
#     bboxes.append(objf.bbox)
#     minr,minc,maxr,maxc=objf.bbox #using the bbox drawn around the melt pool to find width and depth
#     widths.append((maxc-minc)*(0.062)) #using the conversion given in the CDCS comment above the images to convert to mm (1 pixel=0.062 microns)
#     depths.append((maxr-minr)*(0.062))
# dat={'width':widths,'depth':depths,'track':trace,'case':case}
# dfmp=pandas.DataFrame(dat)
# dfmp

In [ ]:

# fig = plt.subplots(1,1)

# for ix, thisrow in datasets_df.iterrows():
    
#     print('Getting image for track {0:,.0f} case {1:}...'.format(thisrow['track'], thisrow['case']))
#     this_image = io.imread(thisrow['url'])

#     print('\tmeasuring melt pool in image...')
#     minr,minc,maxr,maxc = draw_box_opt(this_image).bbox #using the bbox drawn around the melt pool to find width and depth

#     datasets_df.loc[ix, 'melt_pool_width_microns'] = (maxc-minc)*(0.062)
#     datasets_df.loc[ix, 'melt_pool_depth_microns'] = (maxr-minr)*(0.062)
    
#     print('\tshowing marked image...')    
#     fig, ax = plt.subplots(1,1)
    
#     bx = (minc, maxc, maxc, minc, minc)
#     by = (minr, minr, maxr, maxr, minr)
    
#     ax.imshow(this_image[900:1800,800:3400])
#     ax.plot(bx, by, '-b', linewidth=2.5)
#     ax.set_title('Track {0:.0f} case {1:} (width = {2:.1f} µm, depth = {3:.1f} µm)'.format(thisrow['track'], thisrow['case'], datasets_df.loc[ix]['melt_pool_width_microns'], datasets_df.loc[ix]['melt_pool_depth_microns']))
#     plt.show()
    
    
#     #using the conversion given in the CDCS comment above the images to convert to mm (1 pixel=0.062 microns)
# # objf
# #datasets_df.head(2)
# #plt.show()
# print('Done')

In [ ]:
# def show_xml(xml):
#     print('xml tree:')
#     print('---------')
#     root = et.fromstring(xml)
#     for i in range(0, len(root)):
#         if (len(root[i]) <= 1):
#             print('{0:.0f}. {1:}: {2:}'.format(i, root[i].tag, root[i].text))
#         else:
#             print('{0:.0f}. {1:} has {2:.0f} elements!'.format(i, root[i].tag, len(root[i])))
#             for j in range(0, len(root[i])):
#                 if (len(root[i][j]) <= 1):
#                     print('\t{0:.0f}.{1:.0f}. {2:}: {3:}'.format(i, j, root[i][j].tag, root[i][j].text))
#                 else:
#                     print('\t{0:.0f}.{1:.0f}. {2:} has {3:.0f} elements!'.format(i, j, root[i][j].tag, len(root[i][j])))
#                     for k in range(0, len(root[i][j])):
#                         if (len(root[i][j][k]) <= 1):
#                             print('\t\t{0:.0f}.{1:.0f}.{2:.0f}. {3:}: {4:}'.format(i, j, k, root[i][j][k].tag, root[i][j][k].text))
#                         else:
#                             print('\t\t\t{0:.0f}.{1:.0f}.{2:.0f}. {3:} has {4:.0f} elements!'.format(i, j, k, root[i][j][k].tag, len(root[i][j][k])))
#                             for l in range(0, len(root[i][j][k])):
#                                 if (len(root[i][j][k][l]) <= 1):
#                                     print('\t\t\t{0:.0f}.{1:.0f}.{2:.0f}.{3:.0f}. {4:}: {5:}'.format(i, j, k, l, root[i][j][k][l].tag, root[i][j][k][l].text))
#                                 else:
#                                     for m in range(0, len(root[i][j][k][l])):
#                                         if (len(root[i][j][k][l][m]) <= 1):
#                                             print('\t\t\t\t{0:.0f}.{1:.0f}.{2:.0f}.{3:.0f}.{4:.0f}. {5:}: {6:}'.format(i, j, k, l, m, root[i][j][k][l][m].tag, root[i][j][k][l][m].text))
#                                         else:
#                                             print('\t\t\t\t{0:.0f}.{1:.0f}.{2:.0f}.{3:.0f}.{4:.0f}. {5:} has {6:.0f} elements!'.format(i, j, k, l, m, root[i][j][k][l][m].tag, len(root[i][j][k][l][m])))


###### JR: since we are getting and processing images one by one, we no longer need this, keeping for reference
# #Given an xml string like the one returned by CDCS_xml, will return a dataframe of download links for the files on the AMBench
# #page (includes name, description, comments, etc.)
# def CDCS_downloads(xml):
#     root=et.fromstring(xml)
#     result=[]
#     for element in root.iter('downloadURL'):
#         entry={}
#         for e in element.getparent():
#             entry[e.tag]=e.text
#         result.append(entry)
#     df=py.DataFrame.from_dict(result,orient='columns')
#     return df        


# #### draw_box_opt
######## JR: we are now doing these steps in the main body, keeping this function commented out for reference
# # This method (1) crops the melt pool images (to reduce runtime), (2) applies the felzenszwalb segmentation algorithm to the image, (3) removes smaller regions (eliminate some background noise), (4) acquires a table of center coordinates for the remaining regions, (5) measures the distances from centers to the center of the image, (6) finds the index number of the closest region, (7) returns information about the closes region

# # Parameters:
# # - image to be analyzed

# # Returns:
# # - list of RegionProperties related to the melt pool region
# def draw_box_opt(image):
#     cropim=image[900:1800,800:3400] #cropping so that there's less pixels to cover - incredibly slow if left at original size
#     segments=segmentation.felzenszwalb(cropim,scale=270,sigma=0.8,min_size=1000)
#     isolateim=morphology.remove_small_objects(segments,100000)
#     center=measure.regionprops_table(isolateim,properties=['centroid']) #table of information about the center points of regions
#     distances=[]
#     for n in range(len(center['centroid-0'])):
#         distances.append(math.dist([center['centroid-0'][n],center['centroid-1'][n]],[450,1300]))
#     index=distances.index(min(distances)) #Looking for the region closest to the center of the image - should be the melt pool
#     object_features=measure.regionprops(isolateim)
#     return object_features[index] 
